In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch import nn, optim
from torch.utils.data import DataLoader

In [3]:
# 2. 데이터 전처리 및 증강 설정
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),                # 데이터 증강 - 좌우 반전
    transforms.RandomAdjustSharpness(2),              # 데이터 증강 - 선명도 조정
    transforms.ToTensor(),                            # 이미지를 텐서로 변환
    transforms.Normalize((0.5, 0.5, 0.5),              # 정규화 (평균, 표준편차)
                         (0.5, 0.5, 0.5))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),                            # 테스트셋은 증강 없이
    transforms.Normalize((0.5, 0.5, 0.5),
                         (0.5, 0.5, 0.5))
])

# 3. CIFAR-100 데이터셋 로드
trainset = torchvision.datasets.CIFAR100(
    root='./data', train=True, download=True, transform=transform_train)
testset = torchvision.datasets.CIFAR100(
    root='./data', train=False, download=True, transform=transform_test)

trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
testloader = DataLoader(testset, batch_size=64, shuffle=False)

# 4. 개선된 CNN 모델 정의
class ImprovedCNN(nn.Module):
    def __init__(self):
        super(ImprovedCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 3, padding=1)        # conv + BN + ReLU + Pool
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.3)                     # 과적합 방지용 드롭아웃
        self.fc1 = nn.Linear(128 * 8 * 8, 512)             # FC 레이어
        self.fc2 = nn.Linear(512, 100)                     # 100 클래스 출력

    def forward(self, x):
        x = self.pool(torch.relu(self.bn1(self.conv1(x))))
        x = self.pool(torch.relu(self.bn2(self.conv2(x))))
        x = self.dropout(x)
        x = x.view(-1, 128 * 8 * 8)                         # Flatten
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# 5. 모델, 손실함수, 옵티마이저, 스케줄러 정의
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # GPU 사용 여부
model = ImprovedCNN().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

# 6. 훈련 함수
def train(model, trainloader, criterion, optimizer, scheduler, num_epochs=15):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(trainloader):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i % 100 == 99:  # 100 미니배치마다 출력
                print(f"Epoch {epoch+1}, Batch {i+1}, Loss: {running_loss/100:.3f}")
                running_loss = 0.0

        scheduler.step()  # 학습률 감소

# 7. 테스트 함수
def test(model, testloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"\n📊 Accuracy on test set: {100 * correct / total:.2f}%")

# 8. 모델 학습 및 평가
train(model, trainloader, criterion, optimizer, scheduler, num_epochs=15)
test(model, testloader)


100.0%


Epoch 1, Batch 100, Loss: 4.336
Epoch 1, Batch 200, Loss: 3.786
Epoch 1, Batch 300, Loss: 3.550
Epoch 1, Batch 400, Loss: 3.367
Epoch 1, Batch 500, Loss: 3.306
Epoch 1, Batch 600, Loss: 3.198
Epoch 1, Batch 700, Loss: 3.089
Epoch 2, Batch 100, Loss: 2.902
Epoch 2, Batch 200, Loss: 2.861
Epoch 2, Batch 300, Loss: 2.769
Epoch 2, Batch 400, Loss: 2.756
Epoch 2, Batch 500, Loss: 2.705
Epoch 2, Batch 600, Loss: 2.705
Epoch 2, Batch 700, Loss: 2.616
Epoch 3, Batch 100, Loss: 2.480
Epoch 3, Batch 200, Loss: 2.501
Epoch 3, Batch 300, Loss: 2.485
Epoch 3, Batch 400, Loss: 2.500
Epoch 3, Batch 500, Loss: 2.495
Epoch 3, Batch 600, Loss: 2.460
Epoch 3, Batch 700, Loss: 2.435
Epoch 4, Batch 100, Loss: 2.304
Epoch 4, Batch 200, Loss: 2.361
Epoch 4, Batch 300, Loss: 2.306
Epoch 4, Batch 400, Loss: 2.340
Epoch 4, Batch 500, Loss: 2.300
Epoch 4, Batch 600, Loss: 2.323
Epoch 4, Batch 700, Loss: 2.284
Epoch 5, Batch 100, Loss: 2.182
Epoch 5, Batch 200, Loss: 2.234
Epoch 5, Batch 300, Loss: 2.197
Epoch 5,